In [19]:
import pandas as pd
import numpy as np
listings = pd.read_csv('data/listings.csv')
neighbourhoods = pd.read_csv('data/neighbourhoods.csv')
calendar = pd.read_csv('data/calendar.csv')
calendar.head()

,listing_id,date,available,price
0,20872145,2018-09-22,f,NaN
1,20872145,2018-09-21,f,NaN
2,20872145,2018-09-20,f,NaN
3,20872145,2018-09-19,f,NaN
4,20872145,2018-09-18,f,NaN


In [20]:
df1 = calendar
df3 = listings
df2 = df1[df1.available.str[0:] != "f"]
# create panda series for "sum_of_nights" indicating the number of available nights
a = df2.groupby('listing_id').listing_id.count()
df4 = pd.DataFrame({'id':a.index, 'sum_of_nights':a.values})

In [21]:
# Create month column to Calendar.csv
df1['date'] = pd.to_datetime(df1['date'], errors='coerce')
df1['month'] = df1['date'].dt.month
# Adjust data type of price to float
dfa = df1.fillna('$0.00')
dfa['price'] = dfa['price'].str.replace('$', '')
dfa['price'] = dfa['price'].str.replace(',', '')
dfa['price'] = dfa['price'].astype('float64')
dfa

,listing_id,date,available,price,month
0,20872145,2018-09-22,f,0.0,9
1,20872145,2018-09-21,f,0.0,9
2,20872145,2018-09-20,f,0.0,9
3,20872145,2018-09-19,f,0.0,9
4,20872145,2018-09-18,f,0.0,9
5,20872145,2018-09-17,f,0.0,9
6,20872145,2018-09-16,f,0.0,9
7,20872145,2018-09-15,f,0.0,9
8,20872145,2018-09-14,f,0.0,9
9,20872145,2018-09-13,f,0.0,9


In [22]:

clear_nums = {'t':1, 'f':0}
dfa.replace(clear_nums, inplace = True)
mul = []
for i, row in dfa.iterrows():
     mul.append(row[2] * row[3])

In [23]:

dfa['multiple'] = mul
monthly_price = dfa.groupby(['listing_id', 'month']).multiple.mean()
dfb = monthly_price.reset_index()
dfb

,listing_id,month,multiple
0,3781,1,131.451613
1,3781,2,132.142857
2,3781,3,133.064516
3,3781,4,136.066667
4,3781,5,131.451613
5,3781,6,137.333333
6,3781,7,0.000000
7,3781,8,0.000000
8,3781,9,0.000000
9,3781,10,0.000000


In [24]:
lists_id = []
for i in dfb['listing_id']:
    if i not in lists_id:
        lists_id.append(i)
dfc = pd.DataFrame(lists_id, columns = ['id'])
dfc.head()

,id
0,3781
1,5506
2,6695
3,6976
4,8792


In [25]:
def recursive(df, flag):
    price_list = []
    if flag == 13:
        return df
    else:
        for i, row in dfb.iterrows():
            if int(row[1]) == flag:
                price_list.append(row[2])
        df[str(flag)] = price_list
        flag += 1
        return recursive(df, flag) 
df_month = recursive(dfc, 1) 
df_month

,id,1,2,3,4,5,6,7,8,9,10,11,12
0,3781,131.451613,132.142857,133.064516,136.066667,131.451613,137.333333,0.000000,0.000000,0.000000,0.000000,0.000000,137.129032
1,5506,145.000000,145.000000,145.000000,120.333333,145.000000,145.000000,145.000000,145.000000,145.000000,51.451613,145.000000,145.000000
2,6695,169.000000,169.000000,169.000000,123.933333,169.000000,169.000000,169.000000,169.000000,169.000000,32.709677,117.500000,168.354839
3,6976,6.290323,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.967742,35.400000,60.806452
4,8792,100.000000,123.535714,156.741935,160.600000,162.000000,162.000000,162.000000,162.000000,162.000000,26.129032,0.000000,54.870968
5,9273,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,36.290323,7.500000,225.000000
6,9765,229.000000,229.000000,229.000000,292.733333,229.000000,229.000000,229.000000,229.000000,229.000000,132.354839,106.866667,229.000000
7,9824,147.935484,142.000000,233.064516,281.366667,364.000000,259.400000,240.935484,226.225806,228.800000,116.129032,109.266667,181.612903
8,9855,389.000000,389.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,100.258065,389.000000,389.000000
9,9857,322.193548,313.000000,342.000000,462.333333,342.000000,342.000000,342.000000,342.000000,342.000000,323.967742,239.600000,361.161290


In [26]:
# Save to csv files
df_month.to_csv('data/cleansed_calendar.csv', index=False)

In [27]:
# merge two dataframes, based on id, add the new column to 'listings'
df5 = pd.merge(df3, df4, on='id', how='left')
#df6 = pd.merge(df5, df_month, on='id', how='left')
df7 = df5.fillna(0)
# check if there is a null value in id column
df7.id[df7.id == 0].count()
#df1.available.str.contains(r't').sum()
# df1.groupby('listing_id').listing_id.count()
for i,key in enumerate(df7.columns):
    print(i,key)

0 id
1 listing_url
2 scrape_id
3 last_scraped
4 name
5 summary
6 space
7 description
8 experiences_offered
9 neighborhood_overview
10 notes
11 transit
12 access
13 interaction
14 house_rules
15 thumbnail_url
16 medium_url
17 picture_url
18 xl_picture_url
19 host_id
20 host_url
21 host_name
22 host_since
23 host_location
24 host_about
25 host_response_time
26 host_response_rate
27 host_acceptance_rate
28 host_is_superhost
29 host_thumbnail_url
30 host_picture_url
31 host_neighbourhood
32 host_listings_count
33 host_total_listings_count
34 host_verifications
35 host_has_profile_pic
36 host_identity_verified
37 street
38 neighbourhood
39 neighbourhood_cleansed
40 neighbourhood_group_cleansed
41 city
42 state
43 zipcode
44 market
45 smart_location
46 country_code
47 country
48 latitude
49 longitude
50 is_location_exact
51 property_type
52 room_type
53 accommodates
54 bathrooms
55 bedrooms
56 beds
57 bed_type
58 amenities
59 square_feet
60 price
61 weekly_price
62 monthly_price
63 security_

In [28]:
# Delete unimportant features
df8 = df7[df7.columns[[0,1,4,5,6,7,9,17,25,26,37,39,41,48,49,
                       50,51,52,53,54,55,56,58,60,89,91,96]]]
# check if there is any property with 'price == 0'
df8.price[df8.price == float(0)].count()

0

In [29]:
# Delete rows which 'beds == 0'
df9 = df8[df8.beds != 0]

In [30]:
# In column 'price', delete dollar sign($) and comma(,), convert datatype from string to float
df9['price'] = df9['price'].str.replace('$', '')
df9['price'] = df9['price'].str.replace(',', '')
df9['price'] = df9['price'].astype('float64') 
df9['amenities'].value_counts()

/Users/yiwenshen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yiwenshen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/yiwenshen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

{TV,"Cable TV",Internet,"Wireless Internet","Air conditioning",Kitchen,"Pets allowed",Doorman,Gym,Elevator,Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","Carbon monoxide detector",Essentials,Shampoo,"24-hour check-in",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","Self Check-In",Doorman}                                                                                                                                                                                                                                                                                                                                                                                                                                                                           39
{}                                                                                                                                                                                                                                  

In [31]:
heating = []
wifi = []
tv = []
pets = []
Washer_dryer = []
gym = []

def split_amenities(each_loop, string, l):
    if string in each_loop:
        l.append(1)
    else:
        l.append(0)
        
def split(df9):
    for i in df9['amenities']:
        split_amenities(i, 'Heating', heating)
        split_amenities(i, '"Wireless Internet"', wifi)
        split_amenities(i, 'TV', tv)
        split_amenities(i, '"Pets allowed"', pets)
        split_amenities(i, 'Washer,Dryer', Washer_dryer)
        split_amenities(i, 'Gym', gym)

def sum_amenities(df):
    length = []
    for i in df9['amenities']:
        s = i.replace("{", "").replace("}", "").split(",")
        length.append(len(s))
    return length
sum_of_amenities = sum_amenities(df9)
split(df9)


In [32]:
df9['heating'] = heating
df9['wifi'] = wifi
df9['pets_allowed'] = pets
df9['Washer_dryer'] = Washer_dryer
df9['gym'] = gym
df9['sum_of_amenities'] = sum_of_amenities

/Users/yiwenshen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/yiwenshen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yiwenshen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [33]:
df9 = df9[['id', 
'listing_url', 
'name', 
'summary', 
'space', 
'description', 
'neighborhood_overview', 
'picture_url', 
'host_response_time', 
'host_response_rate', 
'street', 
'neighbourhood_cleansed', 
'city', 
'latitude', 
'longitude', 
'is_location_exact', 
'property_type', 
'room_type', 
'accommodates', 
'bathrooms', 
'bedrooms', 
'beds', 
'amenities', 
'heating', 
'wifi', 
'pets_allowed', 
'Washer_dryer', 
'gym', 
'sum_of_amenities', 
'price', 
'instant_bookable', 
'cancellation_policy', 
'sum_of_nights']]

In [34]:
# Save to csv files
df9.to_csv('data/cleansed_listings.csv', index=False)